In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install necessary packages (YOLOv8 and torchsummary for model summary)
!pip install -q ultralytics torchsummary

In [ ]:
# Imports and Device Setup
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchsummary import summary
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from tqdm import tqdm
from PIL import Image
import random
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

In [ ]:
# Load Pretrained YOLOv8s‑CLS v8.0 via ultralytics
# This pulls both the architecture and weights for the ~7 M‑parameter classification network.
from ultralytics import YOLO

# The string 'yolov8s-cls.pt' will download v8.0's official classification checkpoint automatically.
hub_model = YOLO('yolov8s-cls.pt').to(device)
hub_model.model.eval()
print("Successfully loaded YOLOv8s‑CLS via ultralytics."